In [1]:
import numpy as np
import pandas as pd
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc("figure", figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Chapter 10; 데이터 집계와 그룹 연산

In [4]:
# 데이터셋을 분류하고 각 그룹에 집계나 변형 같은 함수를 적용하는 건 데이터 분석 과정에서 무척 중요하다. 

In [5]:
# 데이터를 불러오고 취합해서 하나의 데이터 집합을 준비하고 나면 그룹 통계를 구하거나 가능하다면 피벗테이블을 구해서 보고서를 만들거나 시각화하게 된다. 

In [6]:
# pandas는 데이터 집합을 자연스럽게 나누고 요약할 수 있는 groupby라는 유연한 방법을 제공한다. 

In [7]:
# 관계형 데이터베이스와 SQL이 인기 있는 이유 중 하나는 데이터를 쉽게 합치고 걸러내고 변형하고 집계할 수 있기 때문이다. 

In [8]:
# 하지만 SQL 같은 쿼리문은 그룹 연산에 제약이 있다. 
# 앞으로 살펴보겠지만 파이썬과 pandas의 강력한 표현력을 잘 이용하면 복잡한 그룹 연산도 pandas 객체나 NumPy 배열을 받는 함수의 조합으로 해결할 수 있다. 
# 이 장에서는 다음 내용을 배우게 된다.

In [9]:
# 하나 이상의 키(함수, 배열, DataFrame의 컬럼 이름)를 이용해서 pandas 객체를 여러 조각으로 나누는 방법
# 합계, 평균, 표준편차, 사용자 정의 함수 같은 그룹 요약 통계를 계산하는 방법
# 정규화, 선형회귀, 등급 또는 부분집합 선택 같은 집단 내 변형이나 다른 조작을 적용하는 방법
# 피벗테이블과 교차일람표를 구하는 방법
# 변위치 분석과 다른 통계 집단 분석을 수행하는 방법

In [10]:
# NOTE_ 시계열 데이터의 집계 같은 특수한 groupby 사용 방법을 리샘플링이라고 하는데, 이 내용은 11장에서 따로 다룬다. 

In [11]:
# 10.1 GroupBy 메카닉

In [12]:
# 그룹 연산의 첫 번째 단계에서는 Series, DataFrame 같은 pandas 객체나 아니면 다른 객체에 들어 있는 데이터를 하나 이상의 키를 기준으로 분리한다. 
# 객체는 하나의 축을 기준으로 분리하는데, 예를 들어 DataFrame은 로우(axis=0)로 분리하거나 컬럼(axis=1)으로 분리할 수 있다. 

In [14]:
# 분리하고 나서는 함수를 각 그룹에 적용시켜 새로운 값을 얻어낸다. 마지막으로 함수를 적용한 결과를 하나의 객체로 결합한다. 
# 결과를 담는 객체는 보통 데이터에 어떤 연산을 했는지에 따라 결정된다. 간단한 그룹 연산의 예시를 살펴보자. (그림 10-1) 페이지 386

In [15]:
# 각 그룹의 색인은 다음과 같이 다양한 형태가 될 수 있으며, 모두 같은 타입일 필요도 없다. 
# 1. 그룹으로 묶을 축과 동일한 길이의 리스트나 배열
# 2. DataFrame의 컬럼 이름을 지칭하는 값
# 3. 그룹으로 묶을 값과 그룹 이름에 대응하는 사전이나 Series 객체
# 4. 축 색인 혹은 색인 내의 개별 이름에 대해 실행되는 함수

In [17]:
# 앞 목록에서 마지막 세 방법은 객체를 나눌 때 사용할 배열을 생성하기 위한 방법이라는 것을 기억해야 한다. 
# 먼저 다음과 같이 DataFrame으로 표현되는 간단한 표 형식의 데이터가 있다고 하자. 

In [18]:
# 책 내용을 기반으로 한 코드
df = pd.DataFrame({"key1": ["a", "a", "b", "b", "a"],
                   "key2": ["one", "two", "one", "two", "one"],
                   "data1": np.random.randn(5),
                   "data2": np.random.randn(5)})

In [19]:
df

,key1,key2,data1,data2
0,a,one,-0.204708,1.393406
1,a,two,0.478943,0.092908
2,b,one,-0.519439,0.281746
3,b,two,-0.555730,0.769023
4,a,one,1.965781,1.246435


In [20]:
# 저자가 만든 GitHub에 올라와있는 코드
df = pd.DataFrame({"key1" : ["a", "a", None, "b", "b", "a", None],
                   "key2" : pd.Series([1, 2, 1, 2, 1, None, 1], dtype="Int64"),
                   "data1" : np.random.standard_normal(7),
                   "data2" : np.random.standard_normal(7)})

In [21]:
df

,key1,key2,data1,data2
0,a,1,1.007189,-0.371843
1,a,2,-1.296221,1.669025
2,None,1,0.274992,-0.438570
3,b,2,0.228913,-0.539741
4,b,1,1.352917,0.476985
5,a,<NA>,0.886429,3.248944
6,None,1,-2.001637,-1.021228


In [22]:
# 이 데이터를 key1으로 묶고 각 그룹에서 data1의 평균을 구해보자. 방법 중 하나는 data1에 대해 groupby 메서드로 호출하고 key1 컬럼을 넘기는 것이다. 

In [25]:
# 책 기반 코드- GitHub랑 동일함
grouped = df["data1"].groupby(df["key1"])

In [26]:
grouped

In [27]:
# 이 grouped 변수는 GroupBY 객체다. df["key1"]로 참조되는 중간값에 대한 것 외에는 아무것도 계산되지 않은 객체다. 
# 이 객체는 그룹 연산을 위해 필요한 모든 정보를 가지고 있어서 각 그룹에 어떤 연산을 적용할 수 있게 해준다.

In [28]:
# 예를 들어 그룹별 평균을 구하려면 GroupBy 객체의 mean 메서드를 사용하면 된다. 

In [29]:
grouped.mean()

key1
a    0.199133
b    0.790915
Name: data1, dtype: float64

In [31]:
# 이 예제에서 중요한 점은 데이터(Series 객체)가 그룹 색인에 따라 수집되고 key1 컬럼에 있는 유일한 값으로 색인되는 새로운 Series 객체가 생성된다는 것이다.
# 새롭게 생성된 Series 객체의 색인은 "key1"인데, 그 이유는 DataFrame 컬럼인 df["key1"] 때문이다.

In [32]:
# 만약 여러 개의 배열을 리스트로 넘겼다면 조금 다른 결과를 얻었을 것이다. 

In [33]:
# 책과 GitHub의 코드가 동일함.
means = df["data1"].groupby([df["key1"], df["key2"]]).mean()

In [34]:
means

key1  key2
a     1       1.007189
      2      -1.296221
b     1       1.352917
      2       0.228913
Name: data1, dtype: float64

In [35]:
# 여기서는 데이터를 두 개의 색인으로 묶었고, 그 결과 계층적 색인을 가지는 Series를 얻을 수 있었다. 

In [36]:
means.unstack()

key2,1,2
key1,,
a,1.007189,-1.296221
b,1.352917,0.228913


In [37]:
# 이 예제에서는 그룹의 색인 모두 Series 객체인데, 길이만 같다면 어떤 배열이라도 상관없다. 

In [58]:
# 이하 동일한 코드는 따로 주석하지 않고 다른 코드만 주석을 추가하겠음.

In [60]:
# states = np.array(["Ohio", "California", "California", "Ohio", "Ohio"])
# years = np.array([2005, 2005, 2006, 2005, 2006])
# df["data1"].groupby([states, years]).mean()             # ValueError: Grouper and axis must be same length 발생하여 각주 처리, github 참고하여 수정

In [61]:
states = np.array(["OH", "CA", "CA", "OH", "OH", "CA", "OH"])
years = [2005, 2005, 2006, 2005, 2006, 2005, 2006]
df["data1"].groupby([states, years]).mean()

CA  2005   -0.204896
    2006    0.274992
OH  2005    0.618051
    2006   -0.324360
Name: data1, dtype: float64

In [62]:
# 한 그룹으로 묶을 정보는 주로 같은 DataFrame 안에서 찾게 되는데, 이 경우 컬럼 이름(문자열, 숫자, 혹은 다른 파이썬 객체)을 넘겨서 그룹의 색인으로 사용할 수 있다. 

In [63]:
df.groupby("key1").mean()

,key2,data1,data2
key1,,,
a,1.5,0.199133,1.515376
b,1.5,0.790915,-0.031378


In [71]:
df.groupby("key2").mean()

C:\Users\ahnjh\AppData\Local\Temp\ipykernel_43788\2972670978.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby("key2").mean()


,data1,data2
key2,,
1,0.158365,-0.338664
2,-0.533654,0.564642


In [72]:
df.groupby(["key1", "key2"]).mean()

data1     data2
key1 key2                    
a    1     1.007189 -0.371843
     2    -1.296221  1.669025
b    1     1.352917  0.476985
     2     0.228913 -0.539741

In [73]:
# 위에서 df.groupby("key1").mean() 코드를 보면 key2 컬럼이 결과에서 빠져 있는 것을 확인할 수 있다. 
# 그 이유는 df["key2"]는 숫자 데이터가 아니기 때문인데, 이런 컬럼은 성가신 컬럼이라고 부르며 결과에서 제외시킨다. 
# 기본적으로 모든 숫자 컬럼이 수집되지만 곧 살펴보듯이 원하는 부분만 따로 걸러내는 것도 가능하다. 

In [74]:
# groupby를 쓰는 목적과 별개로, 일반적으로 유용한 GroupBy 메서드는 그룹의 크기를 담고 있는 Series를 반환하는 size 메서드다.

In [75]:
df.groupby(["key1", "key2"]).size()

key1  key2
a     1       1
      2       1
b     1       1
      2       1
dtype: int64

In [88]:
df.groupby("key1", dropna=False).size() 
df.groupby(["key1", "key2"], dropna=False).size() # GitHub에 추가되어 있어서 추가로 작성한 코드다.

key1  key2
a     1       1
      2       1
      <NA>    1
b     1       1
      2       1
NaN   1       2
dtype: int64

In [89]:
df.groupby("key1").count() # GitHub에 추가되어 있어서 추가로 작성한 코드다. 

,key2,data1,data2
key1,,,
a,2,3,3
b,2,2,2


In [90]:
# 그룹 색인에서 누락된 값은 결과에서 제외된다는 것을 기억하자!

In [91]:
# 10.1.1 그룹 간 순회하기

In [92]:
# GroupBy 객체는 이터레이션을 지원하는데, 그룹 이름과 그에 따른 데이터 묶음을 튜플로 반환한다. 다음 예제를 살펴보자. 

In [93]:
for name, group in df.groupby("key1"):
    print(name)
    print(group)

a
  key1  key2     data1     data2
0    a     1  1.007189 -0.371843
1    a     2 -1.296221  1.669025
5    a  <NA>  0.886429  3.248944
b
  key1  key2     data1     data2
3    b     2  0.228913 -0.539741
4    b     1  1.352917  0.476985


In [94]:
# 이처럼 색인이 여럿 존재하는 경우 튜플의 첫 번째 원소가 색인값이 된다. 

In [95]:
for (k1, k2), group in df.groupby(["key1", "key2"]):
    print((k1, k2))
    print(group)

('a', 1)
  key1  key2     data1     data2
0    a     1  1.007189 -0.371843
('a', 2)
  key1  key2     data1     data2
1    a     2 -1.296221  1.669025
('b', 1)
  key1  key2     data1     data2
4    b     1  1.352917  0.476985
('b', 2)
  key1  key2     data1     data2
3    b     2  0.228913 -0.539741


In [96]:
# 당연히 이 안에서 원하는 데이터만 골라낼 수 있다. 한 줄이면 그룹별 데이터를 사전형으로 쉽게 바꿔서 유용하게 사용할 수 있다. 

In [105]:
# pieces = dict(list(df.groupby("key1"))) # 책에 나와있는 코드
pieces = {name: group for name, group in df.groupby("key1")} # Github에 나와있는 코드

In [106]:
pieces["b"]

,key1,key2,data1,data2
3,b,2,0.228913,-0.539741
4,b,1,1.352917,0.476985


In [107]:
# groupby 메서드는 기본적으로 axis=0에 대해 그룹을 만드는데, 다른 축으로 그룹을 만드는 것도 가능하다. 
# 예를 들어 예제로 살펴본 df의 컬럼을 dtype에 따라 그룹으로 묶을 수 있다. 

In [108]:
df.dtypes

key1      object
key2       Int64
data1    float64
data2    float64
dtype: object

In [109]:
# grouped = df.groupby(df.dtypes, axis=1) # 책에 공개된 코드
grouped = df.groupby({"key1": "key", "key2": "key",
                      "data1": "data", "data2": "data"}, axis="columns") # GitHub 코드

In [110]:
# 그룹을 아래처럼 출력해볼 수 있다. 

In [113]:
for dtype, group in grouped:
    print(dtype)
    print(group)  # 책에 공개된 코드

data
      data1     data2
0  1.007189 -0.371843
1 -1.296221  1.669025
2  0.274992 -0.438570
3  0.228913 -0.539741
4  1.352917  0.476985
5  0.886429  3.248944
6 -2.001637 -1.021228
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [114]:
for group_key, group_values in grouped:
    print(group_key)
    print(group_values)   # GitHub 코드

data
      data1     data2
0  1.007189 -0.371843
1 -1.296221  1.669025
2  0.274992 -0.438570
3  0.228913 -0.539741
4  1.352917  0.476985
5  0.886429  3.248944
6 -2.001637 -1.021228
key
   key1  key2
0     a     1
1     a     2
2  None     1
3     b     2
4     b     1
5     a  <NA>
6  None     1


In [115]:
# 10.1.2 컬럼이나 컬럼의 일부만 선택하기

In [116]:
# DataFrame에서 만든 GroupBy 객체를 컬럼 이름이나 담긴 배열로 색인하면 수집을 위해 해당 컬럼을 선택하게 된다. 

In [118]:
df.groupby("key1")["data1"]
df.groupby("key1")[["data2"]]

In [119]:
# 위 코드는 아래 코드에 대한 신택틱 슈거로 같은 결과를 반환한다. 

In [120]:
df["data1"].groupby(df["key1"])
df[["data2"]].groupby(df["key1"])

In [121]:
# 특히 대용량 데이터를 다룰 경우 소수의 컬럼만 집계하고 싶을 때가 종종 있는데, 예를 들어 위 데이터에서 data2 컬럼에 대해서만 평균을 구하고 결과를 DataFrame으로 받고 싶다면 아래와 같이 작성한다. 

In [122]:
df.groupby(["key1", "key2"])[["data2"]].mean()

data2
key1 key2          
a    1    -0.371843
     2     1.669025
b    1     0.476985
     2    -0.539741

In [123]:
# 색인으로 얻은 객체는 groupby 메서드에 리스트나 배열을 넘겼을 경우 DataFrameGroupBy 객체가 되고, 단일 값으로 하나의 컬럼 이름만 넘겼을 경우 SeriesGroupBy 객체가 된다. 

In [124]:
s_grouped = df.groupby(["key1", "key2"])["data2"]

In [125]:
s_grouped

In [126]:
s_grouped.mean()

key1  key2
a     1      -0.371843
      2       1.669025
b     1       0.476985
      2      -0.539741
Name: data2, dtype: float64

In [127]:
# 10.1.3 사전과 Series에서 그룹핑하기

In [128]:
# 그룹 정보는 배열이 아닌 형태로 존재하기도 한다. 다른 DataFrame 예제를 살펴보자. 

In [129]:
people = pd.DataFrame(np.random.standard_normal((5, 5)),
                      columns=["a", "b", "c", "d", "e"],
                      index=["Joe", "Steve", "Wanda", "Jill", "Trey"])

In [130]:
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-0.577087,0.124121,0.302614,0.523772,0.000940
Steve,1.343810,-0.713544,-0.831154,-2.370232,-1.860761
Wanda,-0.860757,NaN,NaN,0.119827,-1.063512
Jill,0.332883,-2.359419,-0.199543,-1.541996,-0.970736
Trey,-1.307030,0.286350,0.377984,-0.753887,0.331286


In [131]:
# 이제 각 컬럼을 나타낼 그룹 목록이 있고, 그룹별로 컬럼의 값을 모두 더한다고 해보자. 

In [132]:
mapping = {"a": "red", "b": "red", "c": "blue",
           "d": "blue", "e": "red", "f" : "orange"}

In [133]:
# 이 사전에서 groupby 메서드로 넘길 배열을 뽑아낼 수 있지만 그냥 이 사전을 groupby 메서드로 넘기자

In [134]:
by_column = people.groupby(mapping, axis=1) # book
by_column.sum()

,blue,red
Joe,0.826386,-0.452026
Steve,-3.201385,-1.230495
Wanda,0.119827,-1.924270
Jill,-1.741538,-2.997272
Trey,-0.375902,-0.689395


In [136]:
by_column = people.groupby(mapping, axis="columns") # GitHub

In [137]:
# Series에 대해서도 같은 기능을 수행하고 있는데, 고정된 크기의 맵이라고 보면 된다. 

In [138]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [139]:
people.groupby(map_series, axis="columns").count()

,blue,red
Joe,2,3
Steve,2,3
Wanda,1,2
Jill,2,3
Trey,2,3


In [140]:
# 10.1.4 함수로 그룹핑하기

In [142]:
# 파이썬 함수를 사용하는 것은 사전이나 Series를 사용해서 그룹을 매핑하는 것보다 좀 더 일반적인 방법이다.
# 그룹 색인으로 넘긴 함수는 색인값 하나마다 한 번씩 호출되며, 반환값은 그 그룹의 이름으로 사용된다. 

In [143]:
# 좀 더 구체적으로 말하자면 좀 전에 살펴본 예제에서 people DataFrame은 사람의 이름을 색인값으로 사용했다.

In [144]:
# 만약 이름의 길이별로 그룹을 묶고 싶다면 이름의 길이가 담긴 배열을 만들어 넘기는 대신 len 함수를 넘기면 된다.

In [145]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.577087,0.124121,0.302614,0.523772,0.000940
4,-0.974148,-2.073069,0.178441,-2.295882,-0.639450
5,0.483052,-0.713544,-0.831154,-2.250405,-2.924273


In [146]:
# 내부적으로는 모두 배열로 변환되므로 함수를 배열, 사전 또는 Series와 섞어 쓰더라도 전혀 문제가 없다.

In [147]:
key_list = ["one", "one", "one", "two", "two"]

In [148]:
people.groupby([len, key_list]).min()

,,a,b,c,d,e
3,one,-0.577087,0.124121,0.302614,0.523772,0.000940
4,two,-1.307030,-2.359419,-0.199543,-1.541996,-0.970736
5,one,-0.860757,-0.713544,-0.831154,-2.370232,-1.860761


In [149]:
# 10.1.5 색인 단계로 그룹핑하기

In [150]:
# 계층적으로 색인된 데이터는 축 색인의 단계 중 하나를 사용해서 편리하게 집계할 수 있는 기능을 제공한다. 다음 예제를 보자.

In [151]:
columns = pd.MultiIndex.from_arrays([["US", "US", "US", "JP", "JP"],
                                    [1, 3, 5, 1, 3]],
                                    names=["cty", "tenor"])

In [152]:
hier_df = pd.DataFrame(np.random.standard_normal((4, 5)), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.349742  0.069877  0.246674 -0.011862  1.004812
1      1.327195 -0.919262 -1.549106  0.022185  0.758363
2     -0.660524  0.862580 -0.010032  0.050009  0.670216
3      0.852965 -0.955869 -0.023493 -2.304234 -0.652469

In [153]:
# 이 기능을 사용하려면 level 예약어를 사용해서 레벨 번호나 이름을 넘기면 된다. 